# Lab 3 - Creating and loading Silver layer

## 3.1 - Create silver layer tables

In [ ]:
--step 3.1.1
DROP TABLE IF EXISTS dbo.Customers;
DROP TABLE IF EXISTS dbo.StockItems;
DROP TABLE IF EXISTS dbo.Invoices;
DROP TABLE IF EXISTS dbo.InvoiceLines;
DROP TABLE IF EXISTS dbo.FactSale;


In [ ]:
--step 3.1.2
CREATE TABLE [dbo].[Customers]
(
    [CustomerID]          [int]           NULL,
    [CustomerName]        [varchar](100)  NULL,
    -- [ContactPerson]       [varchar](50)   NULL,
    [ContactFirstName]    [varchar](50)   NULL,
    [ContactLastName]     [varchar](50)   NULL,
    [PhoneNumber]         [varchar](20)   NULL,
    [WebsiteURL]          [varchar](256)  NULL,
    [DeliveryAddressLine] [varchar](60)   NULL,
    [DeliveryPostalCode]  [varchar](10)   NULL,
    [LastEditedWhen]      [datetime2](6)  NULL
);
GO

CREATE TABLE [dbo].[InvoiceLines]
(
    [InvoiceID]      [int]            NULL,
    [InvoiceLineID]  [int]            NULL,
    [StockItemID]    [int]            NULL,
    [Quantity]       [int]            NULL,
    [UnitPrice]      [decimal](18, 2) NULL,
    [TaxRate]        [decimal](18, 3) NULL,
    [TaxAmount]      [decimal](18, 2) NULL,
    [LineProfit]     [decimal](18, 2) NULL,
    [LastEditedWhen] [datetime2](6)   NULL
);
GO

CREATE TABLE [dbo].[Invoices]
(
    [InvoiceID]      [int]          NULL,
    [InvoiceDate]    [datetime2](6) NULL,
    [CustomerID]     [int]          NULL,
    [LastEditedWhen] [datetime2](6) NULL
);
GO

CREATE TABLE [dbo].[StockItems]
(
    [StockItemID]     [int]            NULL,
    [StockItemName]   [varchar](100)   NULL,
    [Brand]           [varchar](50)    NULL,
    [PackageTypeName] [varchar](50)    NULL,
    [TaxRate]         [decimal](18, 3) NULL,
    [UnitPrice]       [decimal](18, 2) NULL,
    [Tags]            [varchar](max)   NULL,
    [LastEditedWhen]  [datetime2](6)   NULL
);
GO

CREATE TABLE [dbo].[FactSale]
(
    [InvoiceDate]    [datetime2](6)   NULL,
    [InvoiceID]      [int]            NULL,
    [Quantity]       [int]            NULL,
    [UnitPrice]      [decimal](18, 2) NULL,
    [TaxRate]        [decimal](18, 3) NULL,
    [TaxAmount]      [decimal](18, 2) NULL,
    [LineProfit]     [decimal](18, 2) NULL,
    [CustomerID]     [int]            NULL,
    [StockItemID]    [int]            NULL
);
GO

In [ ]:
--step 3.1.3
SELECT
    SCHEMA_NAME(schema_id) AS SchemaName,
    name AS TableName
FROM sys.tables
WHERE
    SCHEMA_NAME(schema_id) IN ('dbo')
ORDER BY
    SchemaName,
    TableName

## Step 3.2 - Create stored procedures for loading dim data into silver layer

In [ ]:
--step 3.2.1
-- Create functions to derive first and last name

CREATE OR ALTER FUNCTION dbo.GetFirstName (@FullName VARCHAR(100))
RETURNS VARCHAR(50)
AS
BEGIN
    RETURN SUBSTRING(@FullName, 1, CHARINDEX(' ', @FullName) - 1)
END
GO

CREATE OR ALTER FUNCTION dbo.GetLastName (@FullName VARCHAR(100))
RETURNS VARCHAR(100)
AS
BEGIN
    RETURN SUBSTRING(@FullName, CHARINDEX(' ', @FullName) + 1, LEN(@FullName))
END


In [ ]:
--step 3.2.2
-- Validate that scalar udfs have been created
SELECT
    SCHEMA_NAME(obj.schema_id) AS SchemaName,
    obj.name AS FunctionName
FROM sys.objects obj
WHERE obj.type = 'FN' AND SCHEMA_NAME(schema_id) = 'dbo'
ORDER BY
    SCHEMA_NAME(schema_id),
    obj.name


In [ ]:
--step 3.2.3
CREATE OR ALTER PROCEDURE dbo.sp_loadCustomers
AS
BEGIN
    TRUNCATE TABLE [dbo].[Customers];
    INSERT INTO dbo.Customers
    SELECT DISTINCT
        CustomerID,
        CustomerName=LTRIM(RTRIM(CustomerName)),
        ContactFirstName=dbo.GetFirstName(ContactPerson),
        ContactLastName=dbo.GetLastName(ContactPerson),
        PhoneNumber,
        WebsiteURL,
        DeliveryAddressLine,
        DeliveryPostalCode,
        LastEditedWhen
    FROM WWI_Bronze.dbo.Customers;
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadInvoices
AS
BEGIN
    TRUNCATE TABLE [dbo].[Invoices];
    INSERT INTO dbo.Invoices
    SELECT
        [InvoiceID],
        [InvoiceDate],
        [CustomerID],
        [LastEditedWhen]
    FROM WWI_Bronze.dbo.Invoices;
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadInvoiceLines
AS
BEGIN
    TRUNCATE TABLE [dbo].[InvoiceLines];
    INSERT INTO dbo.InvoiceLines
    SELECT
        [InvoiceID],
        [InvoiceLineID],
        [StockItemID],
        [Quantity],
        [UnitPrice],
        ISNULL(TaxRate,0.0) AS [TaxRate],
        ISNULL(TaxAmount,0.0) AS [TaxAmount],
        [LineProfit],
        [LastEditedWhen]
    FROM WWI_Bronze.dbo.InvoiceLines
    WHERE ISNULL(Quantity,0.0) > 0 AND ISNULL(UnitPrice,0.0) > 0 AND ISNULL(TaxRate,0.0) >= 0 AND ISNULL(TaxAmount,0.0) >= 0;
END
GO

CREATE OR ALTER PROCEDURE dbo.sp_loadStockItems
AS
BEGIN
    TRUNCATE TABLE [dbo].[StockItems];
    INSERT INTO dbo.StockItems
    SELECT
        [StockItemID],
        [StockItemName],
        ISNULL(LTRIM(RTRIM([Brand])), null),
        ISNULL(LTRIM(RTRIM([PackageTypeName])), null),
        [TaxRate],
        [UnitPrice],
        [Tags],
        [LastEditedWhen]
    FROM WWI_Bronze.dbo.StockItems;
END
GO


In [ ]:
--step 3.2.4
SELECT
    SCHEMA_NAME(p.schema_id) AS SchemaName,
    p.name AS ProcedureName
FROM
    sys.procedures AS p
WHERE
    SCHEMA_NAME(p.schema_id) = 'dbo'
ORDER BY
    SchemaName, ProcedureName



## Step 3.3 - Create procedures for loading fact data into silver layer

In [ ]:
--step 3.3.1
CREATE OR ALTER PROCEDURE dbo.sp_loadFactSales(@InvoiceYear INT = NULL)
AS
BEGIN
    IF @InvoiceYear IS NOT NULL
    BEGIN
        DELETE dbo.FactSale WHERE YEAR(InvoiceDate) = @InvoiceYear
    END
    ELSE
    BEGIN
        TRUNCATE TABLE dbo.FactSale
    END

    INSERT INTO dbo.FactSale
    SELECT
        i.InvoiceDate,
        i.InvoiceID,
        il.Quantity,
        il.UnitPrice,
        il.TaxRate,
        il.TaxAmount,
        il.LineProfit,
        i.CustomerID,
        il.StockItemID
    FROM dbo.Invoices AS i
    INNER JOIN dbo.InvoiceLines AS il
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN dbo.StockItems AS si
        ON il.StockItemID = si.StockItemID
    INNER JOIN dbo.Customers AS c
        ON i.CustomerID = c.CustomerID
    WHERE YEAR(i.InvoiceDate) = ISNULL(@InvoiceYear, YEAR(i.InvoiceDate));
END
GO

In [ ]:
--step 3.3.2
SELECT
    SCHEMA_NAME(p.schema_id) AS SchemaName,
    p.name AS ProcedureName
FROM
    sys.procedures AS p
WHERE
    SCHEMA_NAME(p.schema_id) = 'dbo' AND p.name = 'sp_loadFactSales'
ORDER BY
    SchemaName, ProcedureName

### Step 3.4 - Execute procedures to load the data

In [ ]:
--step 3.4.1
DECLARE @CountBeforeLoadCustomers  BIGINT = (SELECT COUNT_BIG(*) FROM dbo.Customers)
DECLARE @CountBeforeLoadStockItems     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.StockItems)
DECLARE @CountBeforeLoadInvoices BIGINT = (SELECT COUNT_BIG(*) FROM dbo.Invoices)
DECLARE @CountBeforeLoadInvoiceLines     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.InvoiceLines)
DECLARE @CountBeforeLoadFactSale     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.FactSale)

EXEC dbo.sp_loadCustomers;
EXEC dbo.sp_loadStockItems;
EXEC dbo.sp_loadInvoices;
EXEC dbo.sp_loadInvoiceLines;
EXEC dbo.sp_loadFactSales @InvoiceYear = 2013

SELECT 'dbo'   AS SchemaName, 'Customers'       AS TableName, FORMAT(@CountBeforeLoadCustomers,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.Customers        UNION ALL
SELECT 'dbo'   AS SchemaName, 'StockItems'    AS TableName, FORMAT(@CountBeforeLoadStockItems,  'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.StockItems     UNION ALL
SELECT 'dbo'   AS SchemaName, 'Invoices'   AS TableName, FORMAT(@CountBeforeLoadInvoices, 'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.Invoices    UNION ALL
SELECT 'dbo'   AS SchemaName, 'InvoiceLines'       AS TableName, FORMAT(@CountBeforeLoadInvoiceLines,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.InvoiceLines    UNION ALL
SELECT 'dbo'   AS SchemaName, 'FactSale'       AS TableName, FORMAT(@CountBeforeLoadFactSale,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.FactSale
ORDER BY
    SchemaName,
    TableName